In [1]:
import pydrake

# 二次规划

$$
\begin{aligned}
    \min_x \quad & 0.5 x^TQx + b^Tx + c\\
    \text{s.t.} \quad & Ex \leq f
\end{aligned}
$$

其中 $Q$ 半正定。drake 支持一些二次规划的求解器，包括 OSQP, SCS, Gurobi, MOSEK™ 等，请参阅 [Doxygen 文档]( https://drake.mit.edu/doxygen_cxx/group__solvers.html) 查看完整列表。注意，一些商业解决方案（如 Gurobi 和 MOSEK™) 未包含在预编译的 drake 二进制文件中. 关于求解优化问题相关的所有函数，请参考 [Doxygen 文档](https://drake.mit.edu/doxygen_cxx/classdrake_1_1solvers_1_1_mathematical_program.html)。

QP 问题有很多应用，比如 [DifferentialInverseKinematics](https://drake.mit.edu/doxygen_cxx/namespacedrake_1_1manipulation_1_1planner.html#)

## 二次代价函数

In [4]:
from pydrake.solvers import MathematicalProgram, Solve
import numpy as np

prog = MathematicalProgram()
x = prog.NewContinuousVariables(2, "x")

# quadratic cost 1
cost1 = prog.AddQuadraticCost(x[0]**2 + 2*x[0]*x[1] + x[1]**2 + 3*x[0] + 4)
print(cost1)
print(prog.quadratic_costs()[0])

# quadratic cost 2
cost2 = prog.AddQuadraticCost(x[1]*x[1] + 3)
print(f"The number of quadratic costs in prog: {len(prog.quadratic_costs())}")

# quadratic cost 3: x[0]*x[0] + x[0]*x[1] + 1.5*x[1]*x[1] + 2*x[0] + 4*x[1] + 1
cost3 = prog.AddQuadraticCost(Q=[[2, 1], [1, 3]], b=[2, 4], c=1, vars=x)
print(f"cost 3 is {cost3}")

QuadraticCost (4 + 3 * x(0) + (x(0) * (x(0) + x(1))) + (x(1) * (x(0) + x(1))))
QuadraticCost (4 + 3 * x(0) + (x(0) * (x(0) + x(1))) + (x(1) * (x(0) + x(1))))
The number of quadratic costs in prog: 2
cost 3 is QuadraticCost (1 + 2 * x(0) + 4 * x(1) + (x(0) * (x(0) + 0.5 * x(1))) + (x(1) * (0.5 * x(0) + 1.5 * x(1))))


### `AddQuadraticErrorCost`

$$
(x - x_{desired})^TQ(x-x_{desired})
$$

In [5]:
# Adds the cost (x - [1;2])' * Q * (x-[1;2])
cost4 = prog.AddQuadraticErrorCost(Q=[[1, 2],[2, 6]], x_desired=[1,2], vars=x)
print(f"cost4 is {cost4}")

cost4 is QuadraticCost (33 - 10 * x(0) - 28 * x(1) + (x(0) * (x(0) + 2 * x(1))) + (x(1) * (2 * x(0) + 6 * x(1))))


### `Add2NormSquaredCost`

$$
|Ax-b|^2
$$

In [6]:
# Adds the squared norm of (x[0]+2*x[1]-2, x[1] - 3) to the program cost.
cost5 = prog.Add2NormSquaredCost(A=[[1, 2], [0, 1]], b=[2, 3], vars=x)
print(f"cost5 is {cost5}")

cost5 is QuadraticCost (13 - 4 * x(0) - 14 * x(1) + (x(0) * (x(0) + 2 * x(1))) + (x(1) * (2 * x(0) + 5 * x(1))))


## 代价函数是否为凸函数

drake 会计算每个代价函数的 Hessian 矩阵是否为正半定，来检查每是否为凸。drake 支持的凸求解器有 Gurobi / MOSEK™ / OSQP / CLP / SCS 等。但是检查代价函数是否为凸会占用一定的计算资源，如果应用程序要求尽可能快地解决 QP 问题，那么绕过凸检查能大大加快求解速度，因此如果知道代价函数的凸性时，可以在 `AddQuadraticCost` 函数中设置 `is_convex` 标志.

In [8]:
cost7 = prog.AddQuadraticCost(x[0]**2 + 3 * x[1]**2 + x[0] * x[1] + 2 * x[0])
print(f"Is the cost {cost7} convex? {cost7.evaluator().is_convex()}")
cost8 = prog.AddQuadraticCost(x[0]**2 + 3 * x[1]**2 + 4*x[0]*x[1])  # 如果指定 is_convex=True，那么即便不是凸函数也会认定为凸函数
print(f"Is the cost {cost8} convex? {cost8.evaluator().is_convex()}")

cost9 = prog.AddQuadraticCost(x[0]**2 + 4 * x[1]**2 + 3 * x[0]*x[1], is_convex=True)
print(f"Is the cost {cost9} convex? {cost9.evaluator().is_convex()}")

Is the cost QuadraticCost (2 * x(0) + (x(0) * (x(0) + 0.5 * x(1))) + (x(1) * (0.5 * x(0) + 3 * x(1)))) convex? True
Is the cost QuadraticCost ((x(0) * (x(0) + 2 * x(1))) + (x(1) * (2 * x(0) + 3 * x(1)))) convex? True
Is the cost QuadraticCost ((x(0) * (x(0) + 1.5 * x(1))) + (x(1) * (1.5 * x(0) + 4 * x(1)))) convex? True


## 求解算例

In [9]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(3, "x")
prog.AddQuadraticCost(x[0] * x[0] + 2 * x[0] + 3)
# Adds the quadratic cost on the squared norm of the vector
# (x[1] + 3*x[2] - 1, 2*x[1] + 4*x[2] -4)
prog.Add2NormSquaredCost(A = [[1, 3], [2, 4]], b=[1, 4], vars=[x[1], x[2]])

# Adds the linear constraints.
prog.AddLinearEqualityConstraint(x[0] + 2*x[1] == 5)
prog.AddLinearConstraint(x[0] + 4 *x[1] <= 10)
# Sets the bounds for each variable to be within [-1, 10]
prog.AddBoundingBoxConstraint(-1, 10, x)

# Solve the program.
result = Solve(prog)
print(f"optimal solution x: {result.GetSolution(x)}")
print(f"optimal cost: {result.get_optimal_cost()}")

optimal solution x: [-3.90017281e-15  2.50000000e+00 -3.40000000e-01]
optimal cost: 3.3600000000000136
